In [1]:
import sys 
sys.path.append(r"C:\Users\a-sochat\development\Qcodes_Sohail_Clone\Qcodes")

import numpy as np

import qcodes as qc
from qcodes.instrument.parameter import ManualParameter
from qcodes.sweep.sweep import Nest, Zip, Chain, ParameterSweep, ParameterWrapper, FunctionSweep, FunctionWrapper

C:\Users\a-sochat\AppData\Local\Continuum\Anaconda3\envs\qcodes\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [50]:
class Printer:
    def __init__(self, sweep_object): 
        self._table = sweep_object.parameter_table
        self._table_list = self._table.table_list
        
    def __enter__(self): 
        print(self._table)
        return self
    
    def __exit__(self, type, value, traceback): 
        pass

    def __call__(self, result):
        p = [i[0] for i in self._table_list[0]["independent_parameters"]]
        p += [i[0] for i in self._table_list[0]["dependent_parameters"]]
        print(" " + "\t ".join(list([str(result[ip]) for ip in p])))

# Lets introduce a basic sweep object 

In [51]:
x = ManualParameter("x", unit="V")

In [52]:
sweep_object = ParameterSweep(x, lambda: [0, 1])

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

x [V]
 0
 1


We have generated a small 1D coordinate layout with size 2 

# How do we make a 2D sweep? 

In [53]:
y = ManualParameter("y", unit="V")

In [54]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])
sweep_object = Nest([soy, sox])

In [55]:
with Printer(sweep_object) as printer:
    for i in sweep_object:  # X is the inner axis : 
        printer(i)

y [V]	x [V]
 0	 0
 0	 1
 1	 0
 1	 1


This represents a 2D layout of 2x2 

# We can extend this to ND

In [56]:
z = ManualParameter("z", unit="V")

In [57]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])
soz = ParameterSweep(z, lambda: [0, 1])
sweep_object = Nest([soz, soy, sox])

In [58]:
with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

z [V]	y [V]	x [V]
 0	 0	 0
 0	 0	 1
 0	 1	 0
 0	 1	 1
 1	 0	 0
 1	 0	 1
 1	 1	 0
 1	 1	 1


We have created a 2x2x2 layout 

# This is how we can perform a measurement  

In [59]:
m = ManualParameter("m", unit="A")
m.get = lambda: x() ** 2 + y()

In [60]:
x(3)
y(1)
sweep_object = ParameterWrapper(m)

with Printer(sweep_object) as printer:
    for i in sweep_obejct: 
        printer(i)

m [A]
 10


Wrapping a parameter in the "ParameterWrapper" class makes a sweep object which iterates once and returns the "get" value of the parameter. We can use this to create looped measurements. 

In [63]:
sox = ParameterSweep(x, lambda: [0, 1, 2])
soy = ParameterSweep(y, lambda: [0, 1, 3, 4])
measurement = ParameterWrapper(m)
sweep_object =  Nest([soy, sox, measurement])

In [64]:
with Printer(sweep_object) as printer:
    for i in  sweep_object: 
        printer(i)

y [V]	x [V] | m [A]
 0	 0	 0
 0	 1	 1
 0	 2	 4
 1	 0	 1
 1	 1	 2
 1	 2	 5
 3	 0	 3
 3	 1	 4
 3	 2	 7
 4	 0	 4
 4	 1	 5
 4	 2	 8


We see a seperator "|" before "m" in the header. This means that "m" is a dependent parameter, depending on both "x" and "y". In fact, everything after the "|" seperator is a dependent parameter

In [73]:
n = ManualParameter("n", unit="A")
n.get = lambda: x() - y() ** 2 + 16

sox = ParameterSweep(x, lambda: [0, 1, 2])
soy = ParameterSweep(y, lambda: [0, 1, 3, 4])
meas1 = ParameterWrapper(m)
meas2 = ParameterWrapper(n)
sweep_object =  Nest([soy, sox, meas1, meas2])

In [74]:
with Printer(sweep_object) as printer:
    for i in  sweep_object: 
        printer(i)

y [V]	x [V] | m [A]	n [A]
 0	 0	 0	 16
 0	 1	 1	 17
 0	 2	 4	 18
 1	 0	 1	 15
 1	 1	 2	 16
 1	 2	 5	 17
 3	 0	 3	 7
 3	 1	 4	 8
 3	 2	 7	 9
 4	 0	 4	 0
 4	 1	 5	 1
 4	 2	 8	 2


# What happens if we nest a sweep in a measurement?

In [20]:
x(4)

for i in Nest([soy, measurement, sox]): # Notice how "sox" is the inner axis
    pretty_print(i)

y0, m(y0) = 16, x0
y0, m(y0) = 16, x1
y1, m(y1) = 2, x0
y1, m(y1) = 2, x1


This is equivalent to: 

In [21]:
x = 4
for y in [0, 1]: 
    m=x**2 + y
    for x in [0, 1]:
        print(y, m, x)

0 16 0
0 16 1
1 2 0
1 2 1


# Introducing chaining

In [22]:
x = ManualParameter("x")
y = ManualParameter("y")

m = ManualParameter("m")
m.get = lambda: y() ** 3 - 0.1
mm = ParameterWrapper(m)

n = ManualParameter("n")
n.get = lambda: x() ** 2 + 2 * y() + 0.2
nn = ParameterWrapper(n)

In [23]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])

In [24]:
for i in Nest([soy, Chain([mm, Nest([sox, nn])])]):  # = y * (m + x * n) = ym + yxn: 
    pretty_print(i)

y0, m(y0) = -0.1
y0, x0, n(y0,x0) = 0.2
y0, x1, n(y0,x1) = 1.2
y1, m(y1) = 0.9
y1, x0, n(y1,x0) = 2.2
y1, x1, n(y1,x1) = 3.2


We have woven together a 1D and 2D loop. Notice that we immediately see that "m" only depends on y and "n" depends on x and y. There is no need to explicitly state this. With will become important when we discuss data sets. The above sweep is equivalent to...

In [25]:
for y in [0, 1]: 
    m = y ** 3 - 0.1
    print(y, m)
    for x in [0, 1]: 
        n = x ** 2 + 2 * y + 0.2
        
        print(y, x, n)

0 -0.1
0 0 0.2
0 1 1.2
1 0.9
1 0 2.2
1 1 3.2


Note how the above loop is different from... 

In [26]:
x = ManualParameter("x")
y = ManualParameter("y")

m = ManualParameter("m")
m.get = lambda: y() ** 3 - 0.1
mm = ParameterWrapper(m)

n = ManualParameter("n")
n.get = lambda: x() ** 2 + 2 * y() + 0.2
nn = ParameterWrapper(n)

sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])

for i in Nest([soy, mm, sox, nn]):  # = y * m * x * n 
    pretty_print(i)

y0, m(y0) = -0.1, x0, n(y0,x0) = 0.2
y0, m(y0) = -0.1, x1, n(y0,x1) = 1.2
y1, m(y1) = 0.9, x0, n(y1,x0) = 2.2
y1, m(y1) = 0.9, x1, n(y1,x1) = 3.2


This is equivalent to...

In [27]:
for y in [0, 1]: 
    m = y ** 3 - 0.1
    for x in [0, 1]: 
        n = x ** 2 + 2 * y + 0.2
        
        print(y, m, x, n)

0 -0.1 0 0.2
0 -0.1 1 1.2
1 0.9 0 2.2
1 0.9 1 3.2


Thus although the same functions are called at the same time in the sweep, the shape of the output data is different

# We can use arbitrary functions instead of parameters as measurements

In [28]:
def measurement_function(): 
    value = np.random.normal(0, 1, (3,))
    return {"measurement": {"unit": "H", "value": value, "independent_parameter": False}}

In [29]:
x = ManualParameter("x")
y = ManualParameter("y")

for i in Nest([ParameterSweep(x, lambda: [0, 1, 2, 3]), FunctionWrapper(measurement_function)]): 
    pretty_print(i)

x0, measurement(x0) = [-1.31800767  0.473498   -1.24730603]
x1, measurement(x1) = [ 0.30383519  0.13854527  0.56536637]
x2, measurement(x2) = [ 0.92044549 -1.79077928  0.02910082]
x3, measurement(x3) = [-0.82430061  1.153619    2.07675759]


# We can also use functions as loop parameters

In [30]:
from qcodes.sweep.sweep import parameter_log_format

In [39]:
t = ManualParameter("z")
t.get = lambda: int(np.random.uniform(0, 100))

def setter(value): 
    # Do some measurement right before setting the parameter
    temperature = t()
    
    x.set(value)
    return_value = parameter_log_format(x, value, setting=True)
    
    return_value.update(parameter_log_format(t, temperature, setting=False))
    return return_value

In [40]:
for i in Nest([FunctionSweep(setter, lambda: [0, 1, 2, 3]), FunctionWrapper(measurement_function)]): 
    pretty_print(i)

z() = 50, x0, measurement(x0) = [-1.1072556   0.19921069 -0.57509861]
z() = 75, x1, measurement(x1) = [-0.72796408 -1.49406078 -1.50131129]
z() = 55, x2, measurement(x2) = [-0.95979363  0.42259293  0.13107023]
z() = 93, x3, measurement(x3) = [-1.07275228 -0.04884078  1.1439181 ]


# Finally, the sweep values need not be a list or an numpy array, but also a generating function

In [ ]:
class Measurement:
    def __init__(self): 
        self._value = self._roll_dice()
    def __call__(self): 
        self._value = self._roll_dice()
        return {"measurement": {"unit": "H", "value": self._value, "independent_parameter": False}}
    def _roll_dice(self): 
        return np.random.normal(0, 1, (3,))
    def value(self): 
        return self._value

measurement_function = Measurement()

# By allowing the sweep values to be a generator, we can create a feed-back loop between the sweep object and the 
# measurement.

def sweep_values(): 
    value = 0.0
    while value < 2.0:
        yield value 
        value = np.sum(measurement_function.value())  # The next step depends on the measurement

def setter(value): 
    return {"dac_channel": {"unit": "H", "value": "{:.3}".format(value), "independent_parameter": True}}

for i in Nest([FunctionSweep(setter, sweep_values), FunctionWrapper(measurement_function)]): 
    pretty_print(i)

We will loop until the sum of the measurement variables equal 2.0 or more. Since measurement values are three stochstics with a N(0, 1) distribution, the distribution of the sum is N(0, sqrt(3)). The probability of finding 2.0 or more: Z = 2/sqrt(3), which is equal to 12.51%. As the next cell shows, the expectation value of the number of iterations is therefore 8.0 iterations

In [ ]:
def f(ni): # The probability of looping exactly ni times 
    p = 0.1251
    return (1 - p)**(ni - 1) * p

np.sum([ni*f(ni) for ni in range(1, 1000)])

Lets see if this is correct 

In [ ]:
count = 0
N = 10000
for _ in range(N):
    count += len(list(Nest([FunctionSweep(setter, sweep_values), FunctionWrapper(measurement_function)])))

print(count/N)

:-)